In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sudachipy import tokenizer
from sudachipy import dictionary
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
%matplotlib inline

In [4]:
#sudachiの設定
tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C

In [6]:
#コメントリストの読みこみ
df=pd.read_csv("../csvfile/comment_list_20210213_191722.csv",header=None)

In [7]:
#コメントが30件以上ある書籍に限定
df["comment_num"]=df.groupby([0])[0].transform("count")
df=df[df["comment_num"]>=30]
#コメントの整形
df[1]=df[1].apply(lambda x:"".join(x.split()))

In [9]:
%%time
#base=pd.DataFrame()
#
#base["title"]=titlecomment.index

#コメントを形態素解析し名詞のみ残す
titlecomment=df.groupby(0,sort=False)[1].apply(list)
comment_preprocess=[]

for title_index in range(len(titlecomment)):
    comments=titlecomment[title_index]
    for comment in comments:
        comment_all=[[m.normalized_form(),m.part_of_speech()[0]] for m in tokenizer_obj.tokenize(comment, mode)]
        words=[]
        for i in comment_all:
            if i[1] == "名詞":# or i[1]=="動詞":
                words.append(i[0])
        comment_preprocess.append(" ".join(words))

CPU times: user 6min 6s, sys: 3.89 s, total: 6min 10s
Wall time: 6min 33s


In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#tfidf
vec_tfidf = TfidfVectorizer(min_df=10,max_df=0.5)
comment_tfidf = vec_tfidf.fit_transform(comment_preprocess)

#lda
lda = LatentDirichletAllocation(n_components=20,random_state=0)
comment_tfidf_lda=lda.fit_transform(comment_tfidf)

In [131]:
comment_tfidf

<30579x8050 sparse matrix of type '<class 'numpy.float64'>'
	with 554393 stored elements in Compressed Sparse Row format>

In [106]:
#書籍毎にベクトルを平均化
df_comment_tfidf_lda=pd.DataFrame(comment_tfidf_lda)
df_comment_tfidf_lda.index=df[0].values
df_comment_tfidf_lda_mean=df_comment_tfidf_lda.groupby(level=0,sort=False).mean()

In [107]:
# クラスタリング
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=10, random_state=0).fit(df_comment_tfidf_lda_mean)

In [108]:
len(kmeans.labels_)

429

In [109]:
bookclass=pd.DataFrame()
bookclass["title"]=df[0].unique()
bookclass["class"]=kmeans.labels_
bookclass["class"]=bookclass["class"].astype(str)

In [110]:
bookclass.head()

,title,class
0,逆ソクラテス,8
1,AX アックス (角川文庫),8
2,「自分だけの答え」が見つかる 13歳からのアート思考,8
3,たゆたえども沈まず (幻冬舎文庫),6
4,夢をかなえるゾウ1,9


In [111]:
import numpy as np
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(df_comment_tfidf_lda_mean)
X_embedded=pd.DataFrame(X_embedded)
bookclass_tsne=pd.concat([bookclass,X_embedded],axis=1)

In [112]:
import plotly.express as px
fig = px.scatter(bookclass_tsne, x=0, y=1, color="class",
                 hover_data=['title'])
fig.show()

In [113]:
#書籍とクラスの対応表作成
d_bookclass={}
for i,j in zip(bookclass["title"],bookclass["class"]):
    d_bookclass[i]=j
bookclass_all=df[0].map(d_bookclass)

In [114]:
#tfidfによる各クラスの特徴抽出
tfidf_feat=pd.DataFrame(comment_tfidf.toarray(),columns=vec_tfidf.get_feature_names())
tfidf_feat.index=bookclass_all
tfidf_class=tfidf_feat.groupby(level=0).mean()

In [129]:
#各クラスの上位20ワードの確認
feature_words=[]
for row in range(tfidf_class.shape[0]):
    feature_word=tfidf_class.iloc[row,:].sort_values(ascending=False).index[:50]
    feature_words.append(feature_word)
feature_chk=pd.DataFrame(feature_words)

In [130]:
feature_chk

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,こと,自分,言葉,内容,勉強,仕事,人生,著者,理解,必要,コロナ,行動,社会,日本,考え方,2020,本当,時間,世界,本書,価値,相手,問題,お金,参考,システム,文章,部分,https,興味,読書,人間,英語,交渉,opac,意味,気持ち,知識,意識,思考,実践,努力,20,マナー,時代,子供,方法,大人,企業,説明
1,こと,自分,主人公,物語,作品,小説,気持ち,言葉,湿地,人生,カイア,共感,描写,感じ,二人,幸せ,一人,人間,世界,少女,最後,自然,孤独,障害,表現,存在,人物,芥川賞,恋愛,パニック,理解,女性,明かり,其々,pms,時代,生活,全て,アイドル,登場,家族,本当,感情,内容,文章,100,他人,相談,病気,ちゃら
2,有川,こと,作品,村上,春樹,料理,小説,品川,映画,制作,自分,沖縄,物語,主人公,映像,感じ,短編,希望,人称,ひろ,最後,人物,単数,気持ち,登場,現場,家族,会社,謝肉祭,過去,広報室,アイドル,家政婦,世界,食べ物,告白,短編集,不思議,ドラマ,久し振り,御飯,詩集,お母さん,草刈り,ビートルズ,内容,クリーム,其々,文章,編集
3,こと,自分,スマホ,仕事,日本,内容,投資,お金,ai,必要,時間,行動,著者,参考,読書,理解,考え方,人生,人間,勉強,社会,情報,ビジネス,本書,知識,企業,子供,実践,方法,具体的,世界,データ,環境,未来,生活,ストレス,時代,思考,フィンランド,デジタル,問題,興味,アウトプット,紹介,コロナ,説明,会社,伯父,意識,全て
4,こと,優子,家族,宗教,主人公,物語,香り,自分,森宮,気持ち,本屋,両親,最後,作品,愛情,幸せ,ちひろ,魔女,子供,感じ,世界,ラスト,小説,人物,不思議,意味,地政学,世界観,本当,喫茶店,登場,一香,描写,其々,映画,言葉,サキ,バトン,日常,周り,時代,忘れ物,内容,人生,2020,写真,対談,普通,感動,二人
5,半沢,こと,認知症,直樹,バター,ミステリー,作品,自分,シリーズ,最後,贈与,tシャツ,事件,物語,感じ,小説,ドラマ,登場,2020,人物,世界,今回,期待,主人公,頭巾,展開,ドーナツ,鑑定士,其々,ファンタジー,一気,女性,本当,読了,先生,模型,描写,民俗学,ストーリー,気持ち,言葉,童話,少女,家族,里佳,内容,倍返し,梶井,美容,葉介
6,作品,こと,ゴッホ,多聞,東野,最後,物語,圭吾,伊坂,人物,番人,展開,主人公,気持ち,千舟,事件,登場,犯人,小説,大田,感動,一気,思い,ストーリー,ミステリー,感じ,黒澤,人間,自分,少年,家族,ホテル,直木賞,祈念,成長,言葉,ラスト,テオ,幸太郎,精神,不思議,鈴香,本当,人生,2020,リカ,最初,伏線,マスカレード,子供
7,作品,シリーズ,こと,sf,最後,横溝,事件,正史,ミステリー,小説,物語,タイムマシン,楽しみ,前作,登場,今回,日岡,探偵,世界,展開,森見,感じ,作家,人物,続編,読了,朝井,2020,主人公,一気,葉村,宇宙,期待,自分,完結,下巻,神話,宮部,東野,犯人,ブルース,依頼,人類,内容,面壁,相変わらず,殺人,笑い,ホーソーン,ストーリー
8,こと,自分,作品,物語,最後,小説,シリーズ,アート,気持ち,感じ,主人公,世界,子供,登場,言葉,2020,人物,本当,人生,伊坂,殺し屋,大人,文章,展開,一気,内容,読了,家族,人間,二人,作家,時代,短編,タイトル,其々,事件,今回,笑い,女性,美術館,幸せ,思い,ミステリー,意味,エッセー,父親,作者,期待,部分,表現
9,こと,自分,ガネーシャ,人生,自己啓発,物語,シリーズ,最後,作品,幸せ,世界,主人公,登場,気持ち,人物,小説,感じ,52,言葉,家族,ストーリー,今回,本当,ヘルツ,行動,小市民,内容,箇月,弓子,二人,課題,前作,美緒,其々,笑い,ドミノ,人間,教え,時間,恩田,仕事,ホームスパン,三日月,2020,読了,成功,日常,感動,黄な粉,感謝


In [117]:
#タイトルの順位付け
bookrank={}
for num,i in enumerate(df[0].unique()):
    bookrank[i]=num

In [118]:
bookclass["bookrank"]=bookclass["title"].map(bookrank)

In [119]:
bookclass=bookclass.sort_values(by="bookrank")

In [128]:
class_book=pd.DataFrame()
for i in range(10):
    name=f"class_{i}"
    class_book[name]=bookclass[bookclass["class"]==str(i)]["title"].values[:10]
    
    booklist=", ".join(bookclass[bookclass["class"]==str(i)]["title"].values[:20])
    print(f"| {i} |  | {booklist}|")

| 0 |  | 世界一やさしい「やりたいこと」の見つけ方 人生のモヤモヤから解放される自己理解メソッド, あやうく一生懸命生きるところだった, 「育ちがいい人」だけが知っていること, 独学大全 絶対に「学ぶこと」をあきらめたくない人のための55の技法, 2020年6月30日にまたここで会おう 瀧本哲史伝説の東大講義 (星海社新書), なぜ僕らは働くのか-君が幸せになるために考えてほしい大切なこと, 会計クイズを解くだけで財務3表がわかる 世界一楽しい決算書の読み方, 言語化力 言葉にできれば人生は変わる, 話すチカラ, 時間をもっと大切にするための小さいノート活用術, 10代から知っておきたい あなたを閉じこめる「ずるい言葉」, みずほ銀行システム統合、苦闘の19年史 史上最大のITプロジェクト「3度目の正直」, コロナの時代の僕ら, コピーライターじゃなくても知っておきたい 心をつかむ超言葉術, エンド・オブ・ライフ, 面白い! を生み出す妄想術 だから僕は、ググらない。, 死にたいけどトッポッキは食べたい, 1%の努力, 宇宙兄弟とFFS理論が教えてくれる あなたの知らないあなたの強み【自己診断ID付き】, スペースキーで見た目を整えるのはやめなさい ~8割の社会人が見落とす資料作成のキホン|
| 1 |  | 夜明けのすべて, 明け方の若者たち, わたしの美しい庭, ザリガニの鳴くところ, なんで僕に聞くんだろう。, 人は、なぜ他人を許せないのか?, 御社のチャラ男, 首里の馬, 私をくいとめて (朝日文庫), 「具体⇄抽象」トレーニング 思考力が飛躍的にアップする29問 (PHPビジネス新書), これからの男の子たちへ :「男らしさ」から自由になるためのレッスン, パチンコ 上, 愛されなくても別に, 絶対に挫折しない日本史 (新潮新書), 推し、燃ゆ, ゲームの王国 上 (ハヤカワ文庫JA), 帝都地下迷宮, 美しい距離 (文春文庫), ゲームの王国 下 (ハヤカワ文庫JA), 夜の向こうの蛹たち|
| 2 |  | 一人称単数, イマジン?, まずはこれ食べて, アンマーとぼくら (講談社文庫), まだ温かい鍋を抱いておやすみ, 背高泡立草, トラペジウム (角川文庫), 奈落, ミッドナイトスワン (文春文庫), 口福のレシピ, 希望のゆくえ, 二百

In [136]:
#LDA使わないver
df_comment_tfidf_lda=pd.DataFrame(comment_tfidf.toarray())
df_comment_tfidf_lda.index=df[0].values
df_comment_tfidf_lda_mean=df_comment_tfidf_lda.groupby(level=0,sort=False).mean()

kmeans = KMeans(n_clusters=10, random_state=0).fit(df_comment_tfidf_lda_mean)

bookclass=pd.DataFrame()
bookclass["title"]=df[0].unique()
bookclass["class"]=kmeans.labels_
bookclass["class"]=bookclass["class"].astype(str)

X_embedded = TSNE(n_components=2).fit_transform(df_comment_tfidf_lda_mean)
X_embedded=pd.DataFrame(X_embedded)
bookclass_tsne=pd.concat([bookclass,X_embedded],axis=1)

fig = px.scatter(bookclass_tsne, x=0, y=1, color="class",
                 hover_data=['title'])
fig.show()